In [138]:
#算数のcotを生成するプログラム｡多項式の演算がうまくいっていない

import random

def generate_number(integer=True, digits=1):
    if integer:
        if random.choice([True, False]):
            return random.randint(10**(digits-1), 10**digits - 1)
        else:
            return -random.randint(10**(digits-1), 10**digits - 1)
    else:
        if random.choice([True, False]):
            return round(random.uniform(10**(digits-1), 10**digits - 1), random.randint(1, 4))
        else:
            return round(-random.uniform(10**(digits-1), 10**digits - 1), random.randint(1, 4))

def format_number(n, index):
    if index == 0:
        return str(n)
    return f"({n})" if n < 0 else str(n)

def generate_addition(integer=True, digits=1):
    a = generate_number(integer, digits)
    b = generate_number(integer, digits)
    question = f"{format_number(a, 0)} + {format_number(b, 1)} ="
    answer = a + b
    return question, answer, [(a, '+', b)]

def generate_subtraction(integer=True, digits=1):
    a = generate_number(integer, digits)
    b = generate_number(integer, digits)
    question = f"{format_number(a, 0)} - {format_number(b, 1)} ="
    answer = a - b
    return question, answer, [(a, '-', b)]

def generate_multiplication(integer=True, digits=1):
    a = generate_number(integer, digits)
    b = generate_number(integer, digits)
    question = f"{format_number(a, 0)} * {format_number(b, 1)} ="
    answer = a * b
    return question, answer, [(a, '*', b)]

def generate_division(integer=True, digits=1):
    b = generate_number(integer, digits)
    a = b * generate_number(integer, digits)
    question = f"{format_number(a, 0)} / {format_number(b, 1)} ="
    answer = a / b
    return question, answer, [(a, '/', b)]

def generate_mixed_operation(integer=True, num_terms=3, digits=1):
    operations = ['+', '-', '*', '/']
    terms = [generate_number(integer, digits) for _ in range(num_terms)]
    chosen_operations = random.choices(operations, k=num_terms-1)
    
    question = " ".join(f"{format_number(terms[i], i)} {chosen_operations[i]}" for i in range(num_terms-1)) + f" {format_number(terms[-1], num_terms-1)} ="
    
    # 掛け算と割り算を優先的に計算
    expression = []
    for i in range(num_terms - 1):
        if chosen_operations[i] in ['*', '/']:
            if chosen_operations[i] == '*':
                result = terms[i] * terms[i+1]
            else:
                result = terms[i] / terms[i+1]
            expression.append(result)
            terms[i+1] = result
        else:
            expression.append(terms[i])
            expression.append(chosen_operations[i])
    expression.append(terms[-1])

    # 残りの足し算と引き算を左から右へ計算
    answer = expression[0]
    for i in range(1, len(expression), 2):
        if expression[i] == '+':
            answer += expression[i+1]
        else:
            answer -= expression[i+1]
    
    steps = []
    current_value = terms[0]
    for i in range(num_terms - 1):
        if chosen_operations[i] == '*':
            result = current_value * terms[i+1]
        elif chosen_operations[i] == '/':
            result = current_value / terms[i+1]
        elif chosen_operations[i] == '+':
            result = current_value + terms[i+1]
        else:
            result = current_value - terms[i+1]
        steps.append((current_value, chosen_operations[i], terms[i+1]))
        current_value = result
    
    return question, answer, steps

def generate_problems(num_problems):
    problems = []

    for _ in range(num_problems):
        problem_type = random.choice(['addition', 'subtraction', 'multiplication', 'division', 'mixed'])
        integer = random.choice([True, False])
        num_terms = random.randint(2, 7)  # 2から7の項数をランダムに選択
        digits = random.randint(1, 5)  # 1から5桁をランダムに選択
        
        if problem_type == 'addition':
            q, a, steps = generate_addition(integer, digits)
        elif problem_type == 'subtraction':
            q, a, steps = generate_subtraction(integer, digits)
        elif problem_type == 'multiplication':
            q, a, steps = generate_multiplication(integer, digits)
        elif problem_type == 'division':
            q, a, steps = generate_division(integer, digits)
        elif problem_type == 'mixed':
            q, a, steps = generate_mixed_operation(integer, num_terms, digits)

        problems.append((q, steps, a))
    
    return problems

def generate_response(question, steps, answer):
    if len(steps) == 1:
        response = f"問題\n{question}\n\n回答\n\n"
    else:
        response = f"問題\n{question}\n\n回答\n\n計算を行います。\nまず、各部分を計算します。\n\n"
    
    for step in steps:
        if step[1] == '*':
            result = step[0] * step[2]
        elif step[1] == '/':
            result = step[0] / step[2]
        elif step[1] == '+':
            result = step[0] + step[2]
        else:
            result = step[0] - step[2]
        
        if float(int(result)) == result:
            result = int(result)
        
        if len(steps) == 1:
            response += f"{format_number(step[0], 0)} {step[1]} {format_number(step[2], 1)} は {result} です。\n"
        else:
            response += f"    {format_number(step[0], 0)} {step[1]} {format_number(step[2], 1)} は {result} です。\n"

    if len(steps) == 1:
        return response
    
    response += f"\nこれらを合計します。\n{steps[0][0]}"
    for step in steps:
        response += f" {step[1]} {format_number(step[2], 1)}"
    
    # 整数結果のフォーマット
    if float(int(answer)) == answer:
        answer = int(answer)
        response += f" = {answer}\n\nしたがって、結果は {answer} です。"
    else:
        response += f" = {answer:.4f}\n\nしたがって、結果は {answer:.4f} です。"
    
    return response

# 問題を生成して結果を表示
problems = generate_problems(1)
for q, steps, a in problems:
    print(generate_response(q, steps, a))


問題
-7.1 - (-7.2) + (-6.369) + 2.837 * (-6.39) * (-8.1) - 6.8509 =

回答

計算を行います。
まず、各部分を計算します。

    -7.1 - (-7.2) は 0.10000000000000053 です。
    0.10000000000000053 + (-6.369) は -6.268999999999999 です。
    -6.268999999999999 + 2.837 は -3.431999999999999 です。
    -3.431999999999999 * (-18.12843) は 62.216771759999986 です。
    62.216771759999986 * 146.840283 は 9135.928372584805 です。
    9135.928372584805 - 6.8509 は 9129.077472584806 です。

これらを合計します。
-7.1 - (-7.2) + (-6.369) + 2.837 * (-18.12843) * 146.840283 - 6.8509 = -178.0886

したがって、結果は -178.0886 です。
